<a href="https://colab.research.google.com/github/lx-jdar/progra-concurrente/blob/development/TP2_P1_M4_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programación Concurrente - TP3 - Parte 1

Utilizamos 2 prácticas con OpenMP desarrolladas en lenguaje C/C++ nativo en plataforma colab.




En la clase se utiliza dos codigos, uno para la creación de hilos mediante la herramienta OMP en donde cada hilo es creado por el main definido la variable OMP_NUM_THREADS en 7. Cada hilo muestra el mensaje "Hola Mundo..."con su correspondiente numero de hilo del total que se crearon.
El segundo código se trata de determinar la performance de realizar operaciones en forma secuencial versus utilizando la herramienta OpenMP. Se puede observar que la api aprovecha mejor los recursos, resultando en una mejor performance en la ejecución.


---
## 1.1. Ejercicio 1 - Hola Mundo estilo Programación Concurrente


### 1.1.1. Preguntas ejercicio 1:

a) Identifique los 3 componentes de openMP en el ejercicio 1.

El import de la librería omp.h, cabecera "#pragma omp" para definir codigo omp paralelizable y la variable de entorno OMP_NUM_THREADS que indica la cantidad de hilos

b) Realice pruebas modificando la cantidad de hilos (1, 3, 7). Que conclusiones pudo determinar?.

Con 1 thread, no hay problem en el uso de la pantalla para mostrar los mensajes. Pero si hay mas de un hilo, 3, 7. etc, se hace un mal uso de la pantalla, compitiendo los threads para mostrar la información, con lo cual puede ser que los mensajes de los threads se muestren en la misma linea, cuando en realidad se deberían mostrar uno debajo de otro

c) ¿Cuál es la diferencia las formas de asignación *static* y *dynamic* en la cláusula *schedule*?, ¿Qué sucede si utiliza *dynamic* en el código?

Cuando hay referencias repetidas a los mismos objetos, la elección de programación para una construcción for puede determinarse principalmente por características del sistema de memoria, como la presencia y el tamaño de cachés y si los tiempos de acceso a la memoria son uniformes o no uniformes. Estas consideraciones pueden hacer que sea preferible que cada subproceso haga referencia de forma coherente al mismo conjunto de elementos de una matriz en una serie de bucles, incluso si a algunos subprocesos se les asigna relativamente menos trabajo en algunos de los bucles. Esta configuración se puede realizar mediante la programación static con los mismos límites para todos los bucles. Ejemplo:


```
#pragma omp parallel
{
#pragma omp for schedule(static)
  for(i=0; i<n; i++)
    a[i] = work1(i);
#pragma omp for schedule(static)
  for(i=0; i<n; i++)
    if(i>=k) a[i] += work2(i);
}

```
La programación dynamic es adecuada para el caso de una construcción for con las iteraciones que requieren cantidades de trabajo variables, o incluso impredecibles. La programación dynamic se caracteriza por la propiedad de que ningún subproceso espera en la barrera durante más tiempo que el que le toma a otro subproceso ejecutar su iteración final. Este requisito significa que las iteraciones se deben asignar de una en una a los subprocesos a medida que estén disponibles, con sincronización para cada asignación. Ejemplo:

```
#pragma omp parallel for schedule(dynamic)
  for(i=0; i<n; i++) {
    unpredictable_amount_of_work(i);
}
```
En nuestro ejemplo de código no seria apropiado, ya que el thread en su ejecución es predecible, lo cual estaría generando overhead. Si es un codigo que no es deterministico si sería util el uso de dynamic

d) En el for: ¿Que sucede con el valor de j, sí se parametriza como lastprivate?

Las variables especificadas en la variable-list tienen semántica de cláusulas private. La inicialización o construcción ocurre como si se realizara una vez por subproceso, antes de la ejecución del subproceso de la construcción. Es decir, la variable j, tiene el valor que se definió en el hilo maestro. y cada hilo tiene una copia del valor cuando se definió en el principal. Luego terminado los hilos, el hilo maestro sigue conservando la variable como la tenía definida antes de llamar a los hilos hijos


### 1.1.2. Código Lenguaje C

In [ ]:
%%writefile code.cpp

// Hola Mundo desde OpenMP, usando c, ejecutado en Colab.

#include <iostream>
#include <vector>
#include <omp.h>    // Cabecera OpenMP

int main(int argc, char* argv[])
{
  int i,j=0,sum=1;
  std::cout<<"Inicio..."<<std::endl;

  //-----------------------------------------------------------------------------------------------
  // Region paralela
  #pragma omp parallel
  {
        int count_suc = atoi( argv[1] );
        int ventas_x_sucursal[count_suc][DAYS];
        int total = 0;

        srand(SEED);
        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                ventas_x_sucursal[i][j] = (rand() % (LIMIT_SUP - LIMIT_INF + 1)) + LIMIT_INF;
            }
        }

        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                printf("%d \t", ventas_x_sucursal[i][j]);
            }
            printf("\n");
        }
        //
        // Completar código faltante
        //
        #pragma omp parallel for
        for (int i = 0; i < count_suc; i++)
        {
            #pragma omp critical
            for (int j = 0; j < DAYS; j++)
            {
                total+=ventas_x_sucursal[i][j];
            }
        }
        printf("%d\n", total);
  }
  // Region paralela
  //-----------------------------------------------------------------------------------------------

  std::cout<<"Fin..., con j="<<j<<std::endl;
  std::cout<<sum<<std::endl;
}

### 1.1.3 Compilación de Hola Mundo.

In [ ]:
!g++ -o ejercicio_1 -fopenmp code.cpp

In [ ]:
!cat /proc/cpuinfo | grep "cpu cores"

### 1.1.4. Ejecución Hola Mundo en OpenMP.

In [ ]:
%env OMP_NUM_THREADS=7
!./ejercicio_1

---
## 1.2. Ejercicio 2 - Código Faltante
Suponiendo que una empresa desea contabilizar las ventas totales de todas sus sucursales en un periodo de 15 días, se solicita completar la sección de código faltante con el objetivo de realizar dicha tarea de forma concurrente.

Nota: El número de sucursales es un parámetro de entrada.

In [ ]:
%%writefile total_ventas.cpp

#include <omp.h>
#include <stdio.h>
#include <stdlib.h>

#define SEED 4
#define LIMIT_INF 0
#define LIMIT_SUP 100
#define DAYS 15


int main(int argc, char* argv[]){
    if(argv[1])
    {
        int count_suc = atoi( argv[1] );
        int ventas_x_sucursal[count_suc][DAYS];
        int total = 0;

        srand(SEED);
        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                ventas_x_sucursal[i][j] = (rand() % (LIMIT_SUP - LIMIT_INF + 1)) + LIMIT_INF;
            }
        }

        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                printf("%d \t", ventas_x_sucursal[i][j]);
            }
            printf("\n");
        }
        //
        // Completar código faltante
        //
        #pragma omp parallel
        {
            #pragma omp for
            for (int i = 0; i < count_suc; i++)
            {

              for (int j = 0; j < DAYS; j++)
              {
                  total+=ventas_x_sucursal[i][j];
              }
              printf("\n");
            }
        }
        printf("%d\n", total);
    }
    else
    {
        printf("Por favor, ingrese la cantidad de sucursales");
    }
}

In [ ]:
%env OMP_NUM_THREADS=4
!g++ -o total_ventas -fopenmp total_ventas.cpp

In [ ]:
!./total_ventas 5

# ---
## 1.3. Ejercicio 3 - Axpy

La función Axpy es parte de las bibliotecas BLAS [3]. Ella se encarga de realizar la suma de vectores, con la siguiente ecuación:

<center>$Y = \alpha X + Y$</center>

En donde:
> $\alpha$: Es un escala.

> $X$: Es el vector origen.

> $Y$: Es el vector destino.


### 1.3.1 Preguntas Ejercicio 3
a) Ejecute el ejercicio 1.3.2:

    - Con la variable cantidad_N para: 50, 50.000.
    - Con los valores hilos_ 2, 6.

b) Repita la prueba (a), con ciclos 10, 100. ¿Cuál fue la diferencia?

Depende de la cantidad de ciclos, la velocidad de procesamiento en forma secuencial vs paralela puede nos ser tan performante como se espera.
Cuando el tamaño del array es pequeño, no es muy relevante el uso de paralelismo con OMP, los resultados se indica que con un hilo, es suficiente para administrar el tamaño del array. A medida que crece la cantidad de elementos (de 50 a 50.000), la ejecución secuencial empieza a no ser performante, y se produce un ahorro de costo al hacer uso de paralelismo mediante threads.

c) ¿Cómo varía el SpeedUp a medida que aumenta o disminuye la cantidad de hilos?
En algunos casos el SpeedUp es menor a cero y en otros casos dependiendo la cantidad de elementos es mayor a 1.

env: OMP_NUM_THREADS=2 && cant_N=50 && Ciclos=10
SpeedUp : (tiempo Secuencial/tiempo paralelo) : 0.0209808 / 0.0789165 = 0.265861

env: OMP_NUM_THREADS=6 && cant_N=50 && Ciclos=10
SpeedUp : (tiempo Secuencial/tiempo paralelo) : 0.0290871 / 1.94216 = 0.0149767

env: OMP_NUM_THREADS=2 && cant_N=50.000 && Ciclos=10
SpeedUp : (tiempo Secuencial/tiempo paralelo) : 6.30498 / 4.43506 = 1.42162

env: OMP_NUM_THREADS=6 && cant_N=50.000 && Ciclos=10
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 4.89593 / 5.548 = 0.882467

env: OMP_NUM_THREADS=2 && cant_N=50 && Ciclos=100
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0.14782 / 1.47581 = 0.100162

env: OMP_NUM_THREADS=6 && cant_N=50 && Ciclos=100
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0.128984 / 5.09 = 0.0253408

env: OMP_NUM_THREADS=2 && cant_N=50.000 && Ciclos=100
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 52.017 / 47.3042 = 1.09963

env: OMP_NUM_THREADS=6 && cant_N=50.000 && Ciclos=100
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 52.0389 / 56.9491 = 0.913779

Como se observa, con un tamaño de 50.000 es recién que se obtiene una ganancia mayor a 1. Pero superar la cantidad de threads, no necesariamente se traduce en ganancia, tal es el caso de la última ejecución que el SpeedUp dio 0.91

d) ¿Qué sucede con la eficiencia a medida que aumentan la cantidad_N?¿Porqué no llega al valor ideal?.
Cuando aumenta la cantidad de N, el uso de paralelismo para el manejo de elementos, se ve beneficiado, pero no así si se excede el nro de hilos para realizar las tareas. Por lo que se observa, el valor ideal no es alcanzado porque en el valor Ideal mayormente no se considera el overhead, que en el modelo real si existe y es bastante. Si para un numero pequeño de elementos se utiliza muchos hilos, el overhead se consume todo lo que se usa para procesar los elementos de las matrices y no se obtiene mejoras en el tratamiento de los mismos.


### 1.3.2 Código Lenguaje C

In [ ]:
%%writefile code_axpy.cpp
// Axpy con OpenMP, usando c, ejecutado en Colab.

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;


#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

using namespace std;

// ----------------------------------------------------------------------------

void axpy( double alfa, vector<double> &X, vector<double> &Y )
{
  int i;

  #pragma omp parallel for
  for(i=0;i<Y.size();i++)
  {
    Y[i] = alfa * X[i] + Y[i];
  }
}

// ----------------------------------------------------------------------------

int main(int argc, char* argv[])
{
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  float alfa     = atof( argv[1] );
  int cantidad_N = atoi( argv[2] );
  int ciclos     = atoi( argv[3] );

  // --------------------------------------------
  // Defino la memoria de los vectores.

  vector<double> X( cantidad_N );
  vector<double> Y( cantidad_N );

  for (int i=0;i<X.size();i++)
  {
    X[i] = (rand()/(double)RAND_MAX)*0.73;
    Y[i] = (rand()/(double)RAND_MAX)*0.71;
  }

  // --------------------------------------------
  // Realizo la función Axpy con OpenMP.

  TIEMPO_INI( HTH_AXPY_OMP )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Realizo la función Axpy en forma secuencial.

  omp_set_num_threads(1);

  TIEMPO_INI( HTH_AXPY_SEC )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_SEC )

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / omp_get_num_procs();
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

### 1.3.3. Compilación de código C Axpy.

In [ ]:
!g++ -o axpy -fopenmp code_axpy.cpp

### 1.3.4. Ejecución Axpy.

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

Cantidad_N = 50000#@param {type: "number"}
Alfa       = 1.0#@param {type: "number"}
Ciclos     = 100#@param {type: "slider", min: 10, max: 100}
Hilos      = 6#@param {type: "slider", min: 1, max: 10}

# --------------------------------------------

%env OMP_NUM_THREADS=$Hilos
!./axpy $Alfa $Cantidad_N $Ciclos

---
# Medidas de prestaciones en algoritmos paralelos
Las técnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como medida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [6,7].


## SpeedUp
Referencia a la ganancia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = \frac{Tiempo\_Secuencial}{Tiempo\_Paralelo} $</center>



##Eficiencia
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganancia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = \frac{SpeedUp}{Nro\_Procesadores} $</center>


## Coste
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resolución del problema.

<center>$ coste = Nro\_Procesadores \times Tiempo\_Algoritmo$</center>


## Función Overhead
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.

<center>$ Overhead = {Coste}-{Tiempo\_Secuencial} $</center>

---

---
# Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[3] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[6] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[7] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://so-unlam.com.ar/material-clase/HPC/Arquitecturas_de_computadores_avanzadas_(Modulo_1).pdf)

[8] Github OMP Parte 1 [Link](https://github.com/MicrosoftDocs/cpp-docs/blob/main/docs/parallel/openmp/reference/openmp-directives.md#atomic)

[9] Github OMP Parte 2 [Link](https://github.com/MicrosoftDocs/cpp-docs/blob/main/docs/parallel/openmp/reference/openmp-clauses.md#nowait)

[10] Funciones OMP [Link](https://learn.microsoft.com/en-us/cpp/parallel/openmp/reference/openmp-functions?view=msvc-170)

[11] Directivas OMP [Link](https://learn.microsoft.com/es-es/cpp/parallel/openmp/reference/openmp-directives?view=msvc-170)